```shell
Created: 28 Apr 2018
Last update: 08 May 2018
```

# Notes on `mongo`
> **Contents**
1. `mongo` basic
2. `mongo` shell
3. `mongo` schema Modelling

# 0. About import and export CSV

```shell
mongoimport --host localhost --db dbName --collection collectionName --type csv --headerline --file path/file.csv
```

# 1. `mongo` basics

### 1.1 `mongo` data structure –– BSON

**MongoDB** uses **JSON**(JavaScript Object Notation) doucuments in binary-encoded format, which provides many different types and more complicated structure. **BSON** or Binary JSON, extends the JSON model to provide additional data types. 

**Example of JSON**
```JSON
{"menu": {
  "id": "file",
  "value": "File",
  "popup": {
    "menuitem": [
                 {"value": "New", "onclick": "CreateNewDoc()"},
                 {"value": "Open", "onclick": "OpenDoc()"},
                 {"value": "Close", "onclick": "CloseDoc()"}
                ]
  }
}
```
**Compare with BSON**
```BSON
{"_id":123,
 "name":{
     "first":"first name",
     "last":"last name"
 },
 "age":29,
 "Belongings":[
               {"item":"shoes", "value":200},
               {"item":"bicycle", "value":800},
               {"item":"briefcase", "value":1500}
               ]
}
```

## 1.2 Basic CRUD in `mongo`

### 1.2.1 Creation
To create a new database, a new collection, a new document
* `use <database>`: switch to another database. If it doesn't exist, it will initialise a new database.
* `db.createCollection(<collection>)`: initialise a new collection.
* Using variable to temporarily store a document:
```mongo
> var new = {"sid":123,
...            "name":"David",
...            "info":{"year":2018, "mark":90}
...            }
> db.collection.insert(new)
```

* use `mongoimport` in terminal to import json file into database:
```shell
$ mongoimport --host localhost --db taget_DB --collection collectionName --type json --file path/file.json
```

### 1.2.2 Read
* `find()`: return the result of a query. Without given parameter, it will by default return the entire documents in a collection.  
```mongo
> db.collection.find()
```
* `count()`: return the number of documents.
* `sort()`: sort the documents by criteria. `1` for ascending, `-1` for descending.
```mongo
> db.collection.find().sort({"_id":1})
{"_id":123, ...}
{"_id":124, ...}
...
```
* `limit()`: limit the documents to retrieved. No limit if parameter is not given.
* `hasNext()` and `next()`:  
Considering the following codes:
```shell
> var results = db.collection.find({"unit_code":"FIT5148"})
> results.hasNext() #Check whether it is empty.
true
> results.count()
2
> results.next()
{"sid":1, ...}
> results.next()
{"sid":2, ...}
> results.hasNext()
false
> results.count()
2
> results.next()
2018-04-28T22:42:53.058+1000 E QUERY    [thread1] Error: error hasNext: false :
DBQuery.prototype.next@src/mongo/shell/query.js:305:1
@(shell):1:1
```
Now we can see that the `results` variable stores the output of the `find()`. It works as a generator in Python  but "pop" the first document out when execute `next()` function and remember where it ends. It is noteworthy to notice that `count()` only retrieve the number of documents in this variable(which seem to be immutable after the variable has been initialised).  
* `print()`: print out the document.
* `pretty()` or `printjson()`: print out in a more readable style.
* `explain()`: it usually take `"allPlansExecution"` as its parameter. More infomation in **Indexing**.

### 1.2.3 Update  
[Bulk Write](https://docs.mongodb.com/manual/core/bulk-write-operations/)  
To update document(s) of a collection
* `update()`:
* `updateOne()`:
* `updateMany()`:
* `replaceOne()`:
* Another way to populate data: Run JavaScript in terminal

### 1.2.4 Deletion
* `db.<collection>.deleteOne({filter})`: 
* `db.<collection>.deleteMany({filter})`: 
* `db.<collection>.remove()`: delete documents by query criteria.
* `db.<collection>.drop()`: delete the entire **collection**.
* `db.dropDatabase()`: drop the current **database**.

# 2. `mongo` shell 

Date - 28 Apr 2018

Run `JavaScript` in `mongo` shell
---
Let's take a `JavaScript` file as example:  
> `example.js`
```javascript
db = connect("localhost:27017/another") //Specify which database to run
db.evaluation.insertMany(...)
```

Then, we can run this `JavaScript` file in terminal with `mongo` shell:
```
$ mongo example.js
```
It works exactly the same as we are working in the `mongo` shell
```shell
> use another
> db.evaluation.insertMany(...)
```

---

Date - 29 Apr 2018

Aggregation 
---

**Aggregation pipeline**
```
{$match} ➜ {$project} ➜ {$group} ➜ {$sort}
 filter  ➜   reshape  ➜  sum-up  ➜  order     
```

### 1. Aggregate values & Re-shaping documents
Let's take the following dataset to work with:
```
> db.qoh.find({})
{ "_id" : 1, "type" : 2, "supplier" : "OFF", "qty" : 1000 }
{ "_id" : 2, "type" : 2, "supplier" : "OFF", "qty" : 500 }
{ "_id" : 3, "type" : 1, "supplier" : "ON", "qty" : 200 }
{ "_id" : 4, "type" : 3, "supplier" : "OFF", "qty" : 1000 }
{ "_id" : 5, "type" : 1, "supplier" : "ON", "qty" : 40 }
{ "_id" : 6, "type" : 2, "supplier" : "ON", "qty" : 1000 }
{ "_id" : 7, "type" : 3, "supplier" : "ON", "qty" : 5000 }
```
**Using** `$group`, `$sum`, `$match`:  
<font color=red>**Grouping and Sum up**</font>: let's <font color=red>**group**</font> documents with ***same type***, then respectively <font color=red>**sum**</font> the quantity ***"qty"***.(`$group`, `$sum`)  
<font color=red>**Re-shaping**</font>: Noted that ***"total"*** is a newly created field for storing value of the sum of ***"qty"***. 

```
> db.qoh.aggregate(
...                 [{$group:{_id: "$type", total: {$sum: "$qty"}}}]
...                )
{ "_id" : 3, "total" : 6000 }
{ "_id" : 1, "total" : 240 }
{ "_id" : 2, "total" : 2500 }
```

<font color=red>**Matching**</font>:How about extract document which "supplier" matches "ON".
```
> db.qoh.aggregate(
...                 [{$match:{supplier: "ON"}}]
...                )
{ "_id" : 3, "type" : 1, "supplier" : "ON", "qty" : 200 }
{ "_id" : 5, "type" : 1, "supplier" : "ON", "qty" : 40 }
{ "_id" : 6, "type" : 2, "supplier" : "ON", "qty" : 1000 }
{ "_id" : 7, "type" : 3, "supplier" : "ON", "qty" : 5000 }
```

<font color=red>**Counting**</font>: We can also count how many record in total (`$sum:1`)

```
> db.qoh.aggregate([
...                 {$match:{supplier: "ON"}},
...                 {$group:{_id:"$supplier",total:{$sum:1}}}
...                 ])
{ "_id" : "ON", "total" : 4 }
```

Now, let's combine these command into one. We want to have document which ***"supplier"*** is ***"ON"***, then grouped by their ***"type"*** and sum the quantity ***"qty"***, and do sorting the ***"total"*** in ascending order.
```
> db.qoh.aggregate(
...                 [{$match: {supplier: "ON"}},
...                  {$group: {_id: "$type", total: {$sum: "$qty"}}},
...                  {$sort: {total:1}}]
...                )
{ "_id" : 1, "total" : 240 }
{ "_id" : 2, "total" : 1000 }
{ "_id" : 3, "total" : 5000 }
```

---

### 2. Aggregating Array elements

Let's take the following dataset to work with:
```
> db.FIT.find({}).pretty()
{"_id" : 126, 
 "name" : { "first" : "Peter", "last" : "Chen" },
 "course" : "MBIS", 
 "result" : [{ "unit_code" : "FIT9132", 
               "unit_name" : "Database",
               "semester" : 1, 
               "year" : 2017, 
               "mark" : 99 }]}
{"_id" : 124,
 "name" : { "first" : "Albert", "last" : "Einstein" }, 
 "course" : "MBIS", 
 "result" : [{ "unit_code" : "FIT9131", 
               "unit_name" : "Programming", 
               "semester" : 1, 
               "year" : 2017, 
               "mark" : 90 }, 
             { "unit_code" : "FIT9132",
               "unit_name" : "Database", 
               "semester" : 1, 
               "year" : 2017, 
               "mark" : 90 }]}
{"_id" : 127, 
 "name" : { "first" : "Charles", "last" : "Babbage" }, 
 "course" : "MIT", 
 "result" : [{ "unit_code" : "FIT9132", 
               "unit_name" : "Database", 
               "semester" : 1, 
               "year" : 2017, 
               "mark" : 95 }, 
             { "unit_code" : "FIT9131", 
               "unit_name" : "Programming", 
               "semester" : 2, 
               "year" : 2017 }]}
{"_id" : 123,
 "name" : { "first" : "Marie", "last" : "Currie" }, 
 "course" : "MIT", 
 "result" : [{ "unit_code" : "FIT9132", 
               "unit_name" : "Database", 
               "semester" : 1, 
               "year" : 2017, 
               "mark" : 85}, 
             { "unit_code" : "FIT9131", 
               "unit_name" : "Programming", 
               "semester" : 2, 
               "year" : 2017, 
               "mark" : 99 }]}
{"_id" : 125, 
 "name" : { "first" : "Gordon", "last" : "Bell" }, 
 "course" : "MBIS", 
 "result" : [{ "unit_code" : "FIT9132", 
               "unit_name" : "Database", 
               "semester" : 2, 
               "year" : 2017 }]}
{"_id" : 128, 
 "name" : { "first" : "Peter", "last" : "Chen" }, 
 "course" : "MBIS", 
 "result" : [{ "unit_code" : "FIT9132", 
               "unit_name" : "Database", 
               "semester" : 1, 
               "year" : 2017, 
               "mark" : 99 }]}
```

First we use `$unwind` operator to change array elements into serval single documents

```
> db.FIT.aggregate({$unwind:"$result"}).pretty()
```

It means that the "result" array now are **separate**, and **each** document stored in this has been **distributed**. 

```
{"_id" : 127, 
 "name" : { "first" : "Charles", "last" : "Babbage" }, 
 "course" : "MIT", 
 "result" : [{ "unit_code" : "FIT9132", 
               "unit_name" : "Database", 
               "semester" : 1, 
               "year" : 2017, 
               "mark" : 95 }, 
             { "unit_code" : "FIT9131", 
               "unit_name" : "Programming", 
               "semester" : 2, 
               "year" : 2017 }]}
➜
{"_id" : 127,
 "name" : { "first" : "Charles", "last" : "Babbage" },
 "course" : "MIT",
 "result" : {"unit_code" : "FIT9132",
             "unit_name" : "Database",
             "semester" : 1,
             "year" : 2017,
             "mark" : 95 }}
{"_id" : 127,
 "name" : { "first" : "Charles", "last" : "Babbage" },
 "course" : "MIT",
 "result" : {
             "unit_code" : "FIT9131",
             "unit_name" : "Programming",
             "semester" : 2,
             "year" : 2017 }}

```

let's use `$unwind` to work with grouping and sorting:
```
> db.FIT.aggregate(
...                [{ $unwind : "$results" },
...                 { $group : {_id:"$_id", average:{$avg:"$results.mark"}} }, 
...                 { $sort : {average : 1} }])
{ "_id" : 125, "average" : null }
{ "_id" : 124, "average" : 90 }
{ "_id" : 123, "average" : 92 }
{ "_id" : 127, "average" : 95 }
{ "_id" : 128, "average" : 99 }
{ "_id" : 126, "average" : 99 }
```
Note that id=125 doesn't have key "mark" and its value in FIT9132, so that the "average" returns null. However, though id=127 doesn't have the "mark" data in FIT9131, it has the "mark" data of FIT9132. In this case, `mongo` will treat it as it only have one document valid for doing the average.

Another Example on this topic:
First, let's take a look at the dataset:
```
> db.evaluation.find({})
{"_id":1, "results":[{"item":"content", "score":9}, {"item":"presentation", "score":6}]}
{"_id":2, "results":[{"item":"content", "score":8}, {"item":"presentation", "score":8}]}
{"_id":3, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":7}]}
{"_id":4, "results":[{"item":"content", "score":9}, {"item":"presentation", "score":8}]}
{"_id":5, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":6}]}
{"_id":6, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":9}]}
{"_id":7, "results":[{"item":"content", "score":null}, {"item":"presentation", "score":9}]}
{"_id":8, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":null}]}
{ "_id":9, "results":[{"item":"content", "score":7}]}
> db.evaluation.aggregate(
...                      [{ $unwind : "$results" }, 
...                       { $group : { _id : "$results.item", 
...                                    total : {$sum:"$$results.score"},
...                                    average : {$avg:"$results.score"}} }, 
...                       { $sort : {average : 1} }])
{ "_id" : "presentation", "total" : 53, "average" : 7.571428571428571 }
{ "_id" : "content", "total" : 61, "average" : 7.625 }

```
It is important that if a **key:value** pairs is missing, either key or value, this will not be accounted during the operation of `$sum` or `$avg`.

$$9 + 8 + 7 + 9 + 7 + 7 + 7 + 7 = 61$$
$$61\div8 = 7.625$$

$$6 + 8 + 7 + 8 + 6 + 9 + 9 = 53$$
$$53\div7\approx7.57$$

---

Operators
---

---

```shell
Update: 11 May 2018
```
### 0. Matching based on BSON Type

The `$type` operator lets you match results based on their BSON type.

![BSON type table](./BSON_type_table.png)

---

### 1. Array Query Operators
* `$all`: Matches arrays that contain all elements specified in the query.
* `$elemMatch`: Selects documents if element in the array field matches the specified conditions.
* `$size`: Selects documents if the array field is a specified size.

Let's take the following dataset as example:
```
> db.evaluation.find({})
{"_id":1, "results":[{"item":"content", "score":9}, {"item":"presentation", "score":6}]}
{"_id":2, "results":[{"item":"content", "score":8}, {"item":"presentation", "score":8}]}
{"_id":3, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":7}]}
{"_id":4, "results":[{"item":"content", "score":9}, {"item":"presentation", "score":8}]}
{"_id":5, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":6}]}
{"_id":6, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":9}]}
{"_id":7, "results":[{"item":"content", "score":null}, {"item":"presentation", "score":9}]}
{"_id":8, "results":[{"item":"content", "score":7}, {"item":"presentation", "score":null}]}
{ "_id":9, "results":[{"item":"content", "score":7}]}
```
If we want to find documents which elementes match the query criteria, let says, "score" value within "results" array equals to 8.  
Using `$all` to do a **fuzzy search**:
```mongo
> db.evaluation.find({"results.score":{$all:[8]}})
{"_id":2, "results":[{"item":"content", "score":8}, {"item":"presentation", "score":8}]}
{"_id":4, "results":[{"item":"content", "score":9}, {"item":"presentation", "score":8}]}
```

Or, equals to 8 and 9

```mongo
> db.evaluation.find({"results.score":{$all:[8,9]}})
{"_id":4, "results":[{"item":"content", "score":9}, {"item":"presentation", "score":8}]}
```

We can also do a **exact-matched search** by `$elemMatch`:

```mongo
> db.evaluation.find({results:{$elemMatch:{"item":"content","score":8}}})
{"_id":2, "results":[{"item":"content", "score":8}, {"item":"presentation", "score":8}]}
```

As to array, every non-null array has a **size**, we can use `$size` to narrow down the search range:

```
> db.evaluation.find({results:{$size:1}})
{"_id":9, "results":[{"item":"content", "score":7}]}
```

---

```shell
Update: 08 May 2018
```
### 2. Text Search

### 2.1 Intro to RegEx in `mongo` shell

* `$regex`: regular expression for string pattern matching

There are two ways to use regular expression in `mongo` shell:
1. ```db.collection.find({"field":/regex/i})```
2. ```db.collection.find({"field":{$regex:"pattern", $options:"$i"}})```

**Note**
* `/regex/` is shorthand for regex pattern, `i` indicates the matching is no case sensitive (vice versa).
* `$regex:"pattern"` works the same as above, and `$option:"$i"` is for case sensitive. (Careful with the key-value pair: value is a **string**)

### 2.2 Create text index

let's look at these data stored in `FIT_COMPLEX`:
```
{
  "sid": 123, 
  "name": { "first": "Marie", "last": "Currie" }, 
  "course": "MIT", 
  "result": [{"unit_code": "FIT9132",
              "unit_name": "Database",
              "synopsis":"This unit will introduce the concept of data management in an organis ation through relational database technology.",
              "semester": 1,
              "year": [2017, 2018],
              "mark": 100 },
            { "unit_code": "FIT9131", 
              "unit_name": "Programming",
              "synopsis":"This unit aims to provide students with the basic concepts involved i n the development of well structured software using a programming language.",
              "semester": [1,2],
              "year": [2016, 2017, 2018],
              "mark": 80 }] 
} 
{ 
  "sid": 124,
  "name": { "first": "Albert", "last": "Einstein" },
  "course": "MBIS", 
  "result": [{"unit_code": "FIT9132", 
              "unit_name": "Database",
              "synopsis":"This unit will introduce the concept of data management in an organis ation through relational database technology. Theoretical foundation of relational model, analysis and design, implementation of relational database using SQL will be covered.",
              "semester": 2, 
              "year": [2017],
              "mark": 100 }] 
}
```

let's first have a look at the default index in `FIT_COMPLEX`:

```
> db.FIT_COMPLEX.getIndexes()
[
	{
		"v" : 2,
		"key" : {
			"_id" : 1
		},
		"name" : "_id_",
		"ns" : "fit5148_db.FIT_COMPLEX"
	}
]
```

1\. Create a **text index** on the embeded `"result.synopsis"` field:

```
db.FIT_COMPLEX.createIndex({"result.synopsis":"text"})
```

let's check what we have create:

```
line 21> db.FIT_COMPLEX.getIndexes()
[
	...(omitted)
	{
		"v" : 2,
		"key" : {
			"_fts" : "text",
			"_ftsx" : 1
		},
		"name" : "result.synopsis_text",
		"ns" : "fit5148_db.FIT_COMPLEX",
		"weights" : {
			"result.synopsis" : 1
		},
		"default_language" : "english",
		"language_override" : "language",
		"textIndexVersion" : 3
	}
]
```
Now `MongoDB` inserted this text index into our collection, and **all future documents** that have this ﬁeld will be processed and searched through this index

2\. Using `$text` and `$search` to locate matching data:

```
db.FIT_COMPLEX.find({$text:{$search:<pattern>}})
```

`mongo` will automatically use the text index we have just created to find the matching pattern and return the matching item.

How about we want to sort the output docs by **relevance**: by `$meta`

```
db.FIT_COMPLEX.find({$text:{$search:"data"}}, {score:{$meta:"textScore"}})
```

And we also want to see **at what level of relevance** these data are in ascending order: by `sort()`

Here is an example:

```
> db.FIT_COMPLEX.find({$text:{$search:"data"}},{score:{$meta:"textScore"}, _id:0, sid:1}).pretty().sort({score:{$meta:"textScore"}})
{ "sid" : 123, "score" : 0.5454545454545454 }
{ "sid" : 124, "score" : 0.5208333333333334 }
```

**Note here**: we **limit** the output by naming the fields we want to display, since sometime the matching item with so many subfields are difficult for us to find the result clearly.

---

### 3. Aggregation
![Aggregation Pipline](./aggregation_pipline.png)

**Note:The output of each operation is input to the next operation.**

Aggregation **pipline operations** include the following:
> `$project` — Specify fields to be placed in the output document (projected).  
> `$match` — Select documents to be processed, similar to `find()`.  
> `$limit` — Limit the number of documents to be passed to the next step.  
> `$skip` — Skip a specified number of documents.  
> `$unwind` — Expand an array, generating one output document for each array entry.  
> `$group` — Group documents by a specified key.  
> `$sort` — Sort documents.  
> `$geoNear` — Select documents near a geospatial location.  
> `$out` — Write the results of the pipeline to a collection.  
> `$redact` — Control access to certain data.

Here is an **example** defines an aggregation framework pipeline that consists of a match, a group, and then a sort:
```
db.products.aggregate([ {$match: … }, {$group: … }, {$sort: … } ])
```
This series of operations is illustrated below:  
![aggregation framework pipeline](./aggregation_framework_pipeline.png)

1. The entire collection is passed to the `$match` then selects only certain docs from input
2. The output of `$match` is passed to `$group` which groups by a key to provide new information(sum or avg).
3. The output of `$group` finally is passed to `$sort` where sorted the result in certain criteria.

### 3.1 Basic Operators: `$group`, `$sum`, `$avg`
let's have a look at the following data (some fields have been ommited):
```
> db.new.find({},{_id:0, sid:1, course:1, "result.unit_code":1, yearOfUni:1})
{ "sid" : 128, "course" : "MDS", "result" : [ { "unit_code" : "FIT5202" } ], "yearOfUni" : 3 }
{ "sid" : 129, "course" : "MDS", "result" : [ { "unit_code" : "FIT5202" } ], "yearOfUni" : 3 }
{ "sid" : 130, "course" : "MIT", "result" : [ { "unit_code" : "FIT5136" } ], "yearOfUni" : 3 }
```
How about we want to know the total number of student on each course: by `$group`

```
> db.FIT_COMPLEX.aggregate({$group:{_id:"$course", count:{$sum:1}}})
{ "_id" : "MIT", "count" : 1 }
{ "_id" : "MDS", "count" : 2 }
```

How about the groupby on `unit_code`:

```
> db.FIT_COMPLEX.aggregate({$group:{_id:"$result.unit_code", count:{$sum:1}}})
{ "_id" : [ "FIT5136" ], "count" : 1 }
{ "_id" : [ "FIT5202" ], "count" : 2 }
```

Average on `yearOfUni`?  
```
> db.new.aggregate({$group:{_id:"$course", avg:{$avg:"$yearOfUni"}}})
{ "_id" : "MIT", "avg" : 3 }
{ "_id" : "MDS", "avg" : 3 }
```

### 3.2. Joining 2 collections by `$lookup` (LEFT JOIN)

The syntax of **`$lookup`**:

```shell
db.collection.aggregation([
    { 
        $lookup:{ 
            from: "[foreign collection]", 
            localField : "[local field]", 
            foreignField : <field from the documents of the "from" collection> 
            as : "[key name to appear in result]" 
        }
    }
])
```

* **from**: the foreign collection in the same database to perform the join with.

* **localField**: the ﬁeld of input documents. If an input document does not contain the localField, `$lookup` treats the ﬁeld as 'null' for matching purposes.

* **foreignField**: the ﬁeld from the documents in the from collection. If a document in the from collection does not contain the foreignField, $lookup treats as 'null' for matching purposes.

* **as**: the name of the new array ﬁeld to add to the input documents. The new array ﬁeld contains the matching documents from the from collection.

### Example:  
Collection **`users`**:

```JSON
{
    "_id": <omitted>,
    "sid": 123, 
	"name": { "first": "Marie", "last": "Currie" }
}
{
	"_id": <omitted>,
    "sid": 124, 
	"name": {"first": "Albert","last": "Einstein" }
}
```

Collection **`units`**:

```JSON
{
	"_id": <omitted>,
    "sid": 123, 
	"unit_code": "FIT9132", 
	"unit_name": "Database", 
	"semester": 1, 
	"year": [2017], 
	"mark": 100
}
{
	"_id": <omitted>,
    "sid": 123, 
	"unit_code": "FIT9131", 
	"unit_name": "Programming", 
	"semester": [1,2], 
	"year": [2016], 
	"mark": 80
}
{
	"_id": <omitted>,
    "sid": 124, 
	"unit_code": "FIT9132", 
	"unit_name": "Database", 
	"semester": 2, 
	"year": [2017], 
	"mark": 100
}
```

Hence, let's retrieve users along with `completed units`:

```shell
> db.users.aggregate({
    $lookup:{
                from:"units",
                localField:"sid",
                foreignField:"sid", 
                as:"completed_units"
    }
  }).pretty()
```

Then, we can have the **output**:

```shell
{
	"_id" : ObjectId("5af1244ad87cca47ac32bb0a"),
	"sid" : 123,
	"name" : {
		"first" : "Marie",
		"last" : "Currie"
	},
	"completed_units" : [
		{
			"_id" : ObjectId("5af124edd87cca47ac32bb59"),
			"sid" : 123,
			"unit_code" : "FIT9131",
			"unit_name" : "Programming",
			"semester" : [1, 2],
			"year" : [2016],
			"mark" : 80
		},
		{
			"_id" : ObjectId("5af124edd87cca47ac32bb5a"),
			"sid" : 123,
			"unit_code" : "FIT9132",
			"unit_name" : "Database",
			"semester" : 1,
			"year" : [2017],
			"mark" : 100
		}
	]
}
{
	"_id" : ObjectId("5af1244ad87cca47ac32bb0b"),
	"sid" : 124,
	"name" : {"first" : "Albert", "last" : "Einstein"},
	"completed_units" : [
		{
			"_id" : ObjectId("5af124edd87cca47ac32bb58"),
			"sid" : 124,
			"unit_code" : "FIT9132",
			"unit_name" : "Database",
			"semester" : 2,
			"year" : [2017],
			"mark" : 100
		}
	]
}

```

Now we can find that `"completed_units"` contains multiple units correspongding to `sid`.  
Or, we can use `$unwind` to **separate document per unit**:

```shell
> db.getCollection('units').aggregate(
[
    {
        $unwind:"$sid"
    },
    {
        $lookup:{
            from:"users",
            localField:"sid",
            foreignField:"sid",
            as:"completed_sid"
            }
    }
]).pretty()
```

Then, we can have the **output**:

```shell
{
	"_id" : ObjectId("5af124edd87cca47ac32bb58"),
	"sid" : 124,
	"unit_code" : "FIT9132",
	"unit_name" : "Database",
	"semester" : 2,
	"year" : [2017],
	"mark" : 100,
	"completed_sid" : [
		{
			"_id" : ObjectId("5af1244ad87cca47ac32bb0b"),
			"sid" : 124,
			"name" : {"first" : "Albert", "last" : "Einstein"}
		}
	]
}
{
	"_id" : ObjectId("5af124edd87cca47ac32bb59"),
	"sid" : 123,
	"unit_code" : "FIT9131",
	"unit_name" : "Programming",
	"semester" : [1, 2],
	"year" : [2016],
	"mark" : 80,
	"completed_sid" : [
		{
			"_id" : ObjectId("5af1244ad87cca47ac32bb0a"),
			"sid" : 123,
			"name" : {"first" : "Marie", "last" : "Currie"}
		}
	]
}
{
	"_id" : ObjectId("5af124edd87cca47ac32bb5a"),
	"sid" : 123,
	"unit_code" : "FIT9132",
	"unit_name" : "Database",
	"semester" : 1,
	"year" : [2017],
	"mark" : 100,
	"completed_sid" : [
		{
			"_id" : ObjectId("5af1244ad87cca47ac32bb0a"),
			"sid" : 123,
			"name" : {"first" : "Marie", "last" : "Currie"}
		}
	]
}
```

---

```shell
update: 12 May 2018
```

## 3.3 Request fields by `$project`

The `$project` stage has the following prototype form:
```
{ $project: { <specification(s)> } }
```
It can specify the inclusion of fields:  
1. suppression of the _id field:  
```
{ $project: { _id: 0, ... }}
```
2. Include specific fields from embedded documents:  
**Note**: embedded document should be surrounded by quotation mark "".
```
{ $project: {"embDoc.field":1}}
```

3. Addition of new fields:   
```
{ $project: { <field>: {} }}
```
4. Resetting of the values of existing fields. 

5. Specify the exclusion of fields.  
**Note**: if you exclude fields, you cannot also specify the inclusion of fields, reset the value of existing fields, or add new fields.

# 4. Schema Modelling